In [16]:
import pathlib
from sqlalchemy.orm import declarative_base, sessionmaker, registry
from sqlalchemy import Column, String, DateTime, Integer, PickleType, Table, ForeignKey, create_engine
from sqlalchemy.ext.mutable import MutableList
import datetime

In [17]:
BASE_DIR = pathlib.Path().absolute()

# Create a connection string to the database
connection_string = "sqlite:///" + pathlib.Path.joinpath(BASE_DIR, "routing.db").as_posix()
connection_string


# Create engine that will be used to file to the SQLite database. Echo allows us to bypass comments produced by SQL
engine = create_engine(connection_string, echo=True)

# Create a sessionmaker class which we use to populate each individual table.
session = sessionmaker()

# Create mapper registry
mapper_registry = registry()

In [18]:
@mapper_registry.mapped
class Visit:
    __table__ = Table(
        "Visit",
        mapper_registry.metadata,
        Column("id", Integer, primary_key=True, unique=True),
        Column("pat_id", Integer),
        Column("clin_id", Integer, nullable=True),
        Column("exp_date", DateTime, nullable=True),
        Column("status", String, nullable=True),
        Column("time_earliest", DateTime, nullable=True),
        Column("time_latest", DateTime, nullable=True),
        Column("visit_priority", String, nullable=True),
        Column("visit_complexity", String, nullable=True),
        Column("skill_list", MutableList.as_mutable(PickleType), nullable=True),
        Column("discipline", String, nullable=True),
        Column("cancel_reason", String, nullable=True)       
    )

    _c_visit_complexity = ("simple", "routine", "complex")
    _c_visit_priority = ("green", "amber", "red")
    _c_sched_status = ("unassigned", "assigned", "no show", "cancelled")
    _c_cancel_reason = ("clinician unavailable", "patient unavailable", "no longer needed", "expired", "system action")

    def __init__(self, pat_id=10000, clin_id=None, status=1, sched_status="unscheduled", time_earliest="2000", time_latest="2200",
                 exp_date="22/12/2020", visit_complexity=_c_visit_complexity[1], visit_priority=_c_visit_priority[0],
                 skill_list=[1, 2, 3], discipline="", **kwargs):
        """Initializes a new request and links with pat_id. It contains the following attributes:
            req_id, pat_id, name, status, the earliest time, latest time, sched status, and cancel_reason"""
        self._id = 10000
        self.exp_date = datetime.datetime.strptime(exp_date, "%d/%m/%Y")
        self.pat_id = pat_id
        self.clin_id = clin_id
        self._name = "Visit" + str(self._id)
        self.status = status
        self.time_earliest = datetime.datetime.strptime(time_earliest, "%H%M")
        self.time_latest = datetime.datetime.strptime(time_latest, "%H%M")
        self.visit_priority = visit_priority
        self.visit_complexity = visit_complexity
        self.skill_list = skill_list
        self.discipline = discipline
        self._cancel_reason = None
        self._sched_status = sched_status

In [21]:
def create_db(engine, base):
    base.metadata.create_all(engine)

In [22]:
create_db(engine, mapper_registry)

2022-12-20 10:42:18,548 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-20 10:42:18,549 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Visit")
2022-12-20 10:42:18,550 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-20 10:42:18,552 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Visit")
2022-12-20 10:42:18,552 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-20 10:42:18,554 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Visit" (
	id INTEGER NOT NULL, 
	pat_id INTEGER, 
	clin_id INTEGER, 
	exp_date DATETIME, 
	status VARCHAR, 
	time_earliest DATETIME, 
	time_latest DATETIME, 
	visit_priority VARCHAR, 
	visit_complexity VARCHAR, 
	skill_list BLOB, 
	discipline VARCHAR, 
	cancel_reason VARCHAR, 
	PRIMARY KEY (id), 
	UNIQUE (id)
)


2022-12-20 10:42:18,555 INFO sqlalchemy.engine.Engine [no key 0.00082s] ()
2022-12-20 10:42:18,561 INFO sqlalchemy.engine.Engine COMMIT


In [23]:
def create_obj(table, engine, session):
    # Initialize the session. We will bind it to our engine so it knows where to write to
    local_session = session(bind=engine)

    # Create a sample patient
    new_obj = table()

    # Add the row to the table
    local_session.add(new_obj)

    # Commit the row to the table
    local_session.commit()


In [24]:
create_obj(Visit, engine, session)

2022-12-20 10:42:22,711 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-20 10:42:22,714 INFO sqlalchemy.engine.Engine INSERT INTO "Visit" (pat_id, clin_id, exp_date, status, time_earliest, time_latest, visit_priority, visit_complexity, skill_list, discipline, cancel_reason) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2022-12-20 10:42:22,715 INFO sqlalchemy.engine.Engine [generated in 0.00152s] (10000, None, '2020-12-22 00:00:00.000000', 1, '1900-01-01 20:00:00.000000', '1900-01-01 22:00:00.000000', 'green', 'routine', <memory at 0x0000025518B3FE80>, '', None)
2022-12-20 10:42:22,718 INFO sqlalchemy.engine.Engine COMMIT
